In [1]:
from ensemble_boxes import nms, soft_nms, non_maximum_weighted, weighted_boxes_fusion

import os
import shutil
import pandas as pd
import numpy as np

## CSV file load.

In [2]:
# save_paths = [
#     "1",
#     "2",
#     "3",
#     "4",
#     "5",
#     "6",
#     "7",
#     "8",
#     "9",
#     "10",
#     "11",
#     "12"
# ]
save_paths = [f'{i}.csv' for i in range(1, 13)]
lb_scores = [8, 8, 7, 7, 6, 6, 4, 3, 2, 5, 4, 5]

data_dfs = [pd.read_csv(os.path.join("save_dir", "Weighted_Boxes_Fusion", save_path)) for save_path in save_paths]

## Helper function define.

In [3]:
def get_values(predicted):
    predicted = predicted.split(" ")
    
    labels = []
    scores = []
    boxes = []
    
    for i in range(0, len(predicted) - 1, 6):
        label = int(predicted[i])
        score = float(predicted[i + 1])
        box = list(map(lambda x: float(x) / 512, predicted[i + 2:i + 6]))
        
        labels.append(label)
        scores.append(score)
        boxes.append(box)
        
    return boxes, scores, labels

## Weighted boxes fusion.

In [4]:
iou_thr = 0.6
skip_box_thr = 0.0001

In [5]:
wbf_results = []

for idx in range(len(data_dfs[0])):
    boxes_ = []
    scores_ = []
    labels_ = []
    for df_idx in range(len(data_dfs)):
        predicted = data_dfs[df_idx].iloc[idx, 0]
        box, score, label = get_values(predicted)
        boxes_.append(box)
        scores_.append(score)
        labels_.append(label)
        
    wbf_box, wbf_score, wbf_label = weighted_boxes_fusion(boxes_, scores_, labels_, weights=lb_scores, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    wbf_result = [[int(wbf_label_)] + [wbf_score_] + list(map(lambda x: x * 512, wbf_box_)) for wbf_box_, wbf_score_, wbf_label_ in zip(wbf_box, wbf_score, wbf_label)]
    wbf_result = sorted(wbf_result, key=lambda x: x[1], reverse=True)
    wbf_result = sorted(wbf_result, key=lambda x: x[0])
    
    wbf_results.append(wbf_result)

/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.2954540625, 1.0, 0.3533578515625, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.29238939453125, 1.0, 0.35037970703125, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.2859361328125, 1.0, 0.346594609375, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 2.7751923828125e-06, 0.0, 0.030826806640625].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zer

## Save to CSV file.

In [6]:
image_names = data_dfs[0].iloc[:, 1]

prediction_strings = []
file_names = []

for wbf_result, image_name in zip(wbf_results, image_names):
    prediction_string = ""
    for wbf_result_ in wbf_result:
        label = str(wbf_result_[0])
        score = str(wbf_result_[1])
        box = " ".join(list(map(lambda x: str(x), wbf_result_[2:])))
        prediction_string += label + " " + score + " " + box + " "
    
    prediction_strings.append(prediction_string)
    file_names.append(image_name)
    
submission = pd.DataFrame()
submission["PredictionString"] = prediction_strings
submission["image_id"] = file_names
submission.to_csv(os.path.join("save_dir", f"submission_wbf_ensemble.csv"), index=None)
submission.head()

,PredictionString,image_id
0,0 0.02792796492576599 0.07789668440818787 334....,batch_01_vt/0021.jpg
1,0 0.07869534939527512 284.3097229003906 341.06...,batch_01_vt/0028.jpg
2,0 0.07040009647607803 235.73126220703125 461.1...,batch_01_vt/0031.jpg
3,0 0.06808287650346756 295.6249084472656 157.62...,batch_01_vt/0032.jpg
4,0 0.05581524968147278 107.96443176269531 252.4...,batch_01_vt/0070.jpg
